In [1]:
import pandas as pd
import numpy as np
import logging
from datetime import datetime, date
from io import StringIO

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')


In [2]:
# If your modules are in a different directory, adjust sys.path
import sys
sys.path.append('path_to_your_project_directory')

# Import your classes and functions
from data_processing.data_processor import DataProcessor
from data_processing.report_generator import ReportGenerator
from utils.helpers import determine_period
from config.settings import HARD_CODED_DATA

# Since the modules might not be accessible, we can define the classes and functions directly in the notebook


In [36]:
def determine_period(input_date):
    """
    Determines the period identifier for a given date based on the Excel calculation.

    Args:
        input_date (datetime.date or datetime.datetime or str): The input date.

    Returns:
        str or None: The period identifier (e.g., 'P1', 'P2', etc.), or None if the date is invalid.
    """
    # Handle NaT values
    if pd.isna(input_date):
        return None

    # Convert input_date to date object if it's a string
    if isinstance(input_date, str):
        try:
            input_date = datetime.strptime(input_date, "%d/%m/%Y").date()
        except ValueError:
            return None  # Return None if the date string is invalid
    elif isinstance(input_date, datetime):
        input_date = input_date.date()
    elif not isinstance(input_date, date):
        return None  # Return None for any other unexpected type

    # Define the base date (start of P1)
    base_date = date(2020, 1, 1)

    # Calculate the number of days since the base date
    days_since_base = (input_date - base_date).days

    # Handle dates before the base date
    if days_since_base < 0:
        return None

    # Calculate the year difference
    years_since_base = input_date.year - base_date.year

    # Calculate the period within the year
    if input_date.month <= 6:
        period_in_year = 1
    elif input_date.month <= 9:
        period_in_year = 2
    elif input_date.month <= 12:
        period_in_year = 3

    # Calculate the final period number
    period_number = years_since_base * 3 + period_in_year

    return f"P{period_number}"

test_dates = [
    "01/10/2022", "01/04/2020", "01/07/2020", "01/10/2020",
    "01/01/2021", "01/04/2021", "01/07/2021", "01/10/2021",
    "01/01/2022", "01/04/2022", "01/07/2022"
]

for date_str in test_dates:
    period = determine_period(date_str)
    print(f"Date: {date_str}, Period: {period}")

Date: 01/10/2022, Period: P9
Date: 01/04/2020, Period: P1
Date: 01/07/2020, Period: P2
Date: 01/10/2020, Period: P3
Date: 01/01/2021, Period: P4
Date: 01/04/2021, Period: P4
Date: 01/07/2021, Period: P5
Date: 01/10/2021, Period: P6
Date: 01/01/2022, Period: P7
Date: 01/04/2022, Period: P7
Date: 01/07/2022, Period: P8


In [37]:
#    Determines the period identifier for a given date.

def determine_period(input_date):
    """
    Determines the period identifier for a given date.

    The periods are defined based on specific date ranges, following a recurring
    pattern every two years. The function calculates the period number based on
    the input date.

    Args:
        input_date (datetime.date or datetime.datetime or str): The input date.

    Returns:
        str or None: The period identifier (e.g., 'P1', 'P2', etc.), or None if the date is invalid.
    """
    # Handle NaT values
    if pd.isna(input_date):
        return None

    # Convert input_date to date object if it's a string
    if isinstance(input_date, str):
        try:
            input_date = datetime.strptime(input_date, "%d/%m/%Y").date()
        except ValueError:
            return None  # Return None if the date string is invalid
    elif isinstance(input_date, datetime):
        input_date = input_date.date()
    elif not isinstance(input_date, date):
        return None  # Return None for any other unexpected type

    # Define the base date (start of P1)
    base_date = date(2020, 1, 1)

    # Calculate the number of days since the base date
    days_since_base = (input_date - base_date).days

    # Handle dates before the base date
    if days_since_base < 0:
        return None  # or you could return a special period for pre-2020 dates

    # Calculate the number of complete 2-year cycles
    two_year_cycles = days_since_base // 730  # 730 days in 2 years (ignoring leap years for simplicity)

    # Calculate the remaining days within the current 2-year cycle
    days_in_cycle = days_since_base % 730

    # Determine the period within the 2-year cycle
    if days_in_cycle < 181:  # First 6 months (181 days)
        period_in_cycle = 1
    elif days_in_cycle < 273:  # Next 3 months (92 days)
        period_in_cycle = 2
    elif days_in_cycle < 365:  # Next 3 months (92 days)
        period_in_cycle = 3
    elif days_in_cycle < 456:  # Next 3 months (91 days)
        period_in_cycle = 4
    elif days_in_cycle < 547:  # Next 3 months (91 days)
        period_in_cycle = 5
    elif days_in_cycle < 638:  # Next 3 months (91 days)
        period_in_cycle = 6
    else:  # Last 3 months (92 days)
        period_in_cycle = 7

    # Calculate the final period number
    period_number = two_year_cycles * 7 + period_in_cycle

    return f"P{period_number}"

def clean_isin(isin):
    """
    Cleans and standardizes ISIN codes.

    Args:
        isin (str): The ISIN code to clean.

    Returns:
        str: The cleaned and standardized ISIN code.
    """
    return str(isin).strip().upper()


test_dates = [
    "01/10/2022", "01/04/2020", "01/07/2020", "01/10/2020",
    "01/01/2021", "01/04/2021", "01/07/2021", "01/10/2021",
    "01/01/2022", "01/04/2022", "01/07/2022"
]

for date_str in test_dates:
    period = determine_period(date_str)
    print(f"Date: {date_str}, Period: {period}")

Date: 01/10/2022, Period: P10
Date: 01/04/2020, Period: P1
Date: 01/07/2020, Period: P2
Date: 01/10/2020, Period: P3
Date: 01/01/2021, Period: P4
Date: 01/04/2021, Period: P5
Date: 01/07/2021, Period: P6
Date: 01/10/2021, Period: P7
Date: 01/01/2022, Period: P8
Date: 01/04/2022, Period: P8
Date: 01/07/2022, Period: P9


In [39]:
from datetime import datetime, date

def determine_period(input_date):
    """
    Determines the period identifier for a given date based on the Reference Period table.

    Args:
        input_date (datetime.date or datetime.datetime or str): The input date.

    Returns:
        str or None: The period identifier (e.g., 'P1', 'P2', etc.), or None if the date is invalid.
    """
    # Handle NaT values
    if pd.isna(input_date):
        return None

    # Convert input_date to date object if it's a string
    if isinstance(input_date, str):
        try:
            input_date = datetime.strptime(input_date, "%d/%m/%Y").date()
        except ValueError:
            return None  # Return None if the date string is invalid
    elif isinstance(input_date, datetime):
        input_date = input_date.date()
    elif not isinstance(input_date, date):
        return None  # Return None for any other unexpected type

    # Define the base date (start of P1)
    base_date = date(2020, 1, 1)

    # Handle dates before the base date
    if input_date < base_date:
        return None

    # Calculate the year difference
    years_since_base = input_date.year - base_date.year

    # Determine the quarter based on the month
    if input_date.month <= 3:
        quarter = 4
        year = input_date.year - 1
    elif input_date.month <= 6:
        quarter = 1
        year = input_date.year
    elif input_date.month <= 9:
        quarter = 2
        year = input_date.year
    else:
        quarter = 3
        year = input_date.year

    # Calculate the period number
    period_number = (year - base_date.year) * 4 + quarter

    return f"P{period_number}"

# Test the function
test_dates = [
    "01/10/2022", "01/04/2020", "01/07/2020", "01/10/2020",
    "01/01/2021", "01/04/2021", "01/07/2021", "01/10/2021",
    "01/01/2022", "01/04/2022", "01/07/2022", "01/10/2022"
]

for date_str in test_dates:
    period = determine_period(date_str)
    print(f"Date: {date_str}, Period: {period}")

Date: 01/10/2022, Period: P11
Date: 01/04/2020, Period: P1
Date: 01/07/2020, Period: P2
Date: 01/10/2020, Period: P3
Date: 01/01/2021, Period: P4
Date: 01/04/2021, Period: P5
Date: 01/07/2021, Period: P6
Date: 01/10/2021, Period: P7
Date: 01/01/2022, Period: P8
Date: 01/04/2022, Period: P9
Date: 01/07/2022, Period: P10
Date: 01/10/2022, Period: P11


In [27]:
esma_si_df = r"C:\Users\UT3N95\Desktop\Structured_project\_data_\_ESMA_\xml"
trade_source_df = r'C:\Users\UT3N95\Desktop\Structured_project\_data_\_Murex_\trades.xlsx'
trade_source_scope_df = r"C:\Users\UT3N95\Desktop\Structured_project\_data_\_Murex_\trades.xlsx"
esma_threshold_df = r"C:\Users\UT3N95\Desktop\Structured_project\_data_\_ESMA_\sovereign_debt_thresholds.xlsx"



In [28]:
# Instantiate DataProcessor with DataFrames
data_processor = DataProcessor(
    esma_si_df=esma_si_df,
    trade_source_file=trade_source_df,
    trade_source_scope_file=trade_source_scope_df,
    esma_threshold_file=esma_threshold_df
)

# Run data processing
data_processor.process_data()


2024-09-24 12:15:56,189 INFO:All input files are validated and exist.
2024-09-24 12:15:56,223 INFO:Loaded ESMA_Threshold data.
2024-09-24 12:19:45,010 INFO:Loaded Trade_Source and Trade_Source_Scope data.
2024-09-24 12:19:45,012 ERROR:An error occurred during data processing: 'str' object has no attribute 'columns'


AttributeError: 'str' object has no attribute 'columns'

In [29]:
# Access the processed data
processed_trade_source = data_processor.trade_source
processed_trade_source_scope = data_processor.trade_source_scope
result_df = data_processor.result_df
issuer_review = data_processor.issuer_review


In [30]:
# Check the first few rows
print(processed_trade_source.head())
print(processed_trade_source_scope.head())
print(result_df.head())
print(issuer_review.head())

# Check DataFrame info
processed_trade_source.info()
processed_trade_source_scope.info()
result_df.info()
issuer_review.info()


       M_NB M_TRN_STATUS M_TRN_DATE  M_TRN_EXP        PORTFOLIO  \
0  58115165   LIVE       2023-09-01 2052-10-31  NONCORE8          
1  58115398   LIVE       2023-09-01 2032-10-25  ALGO1             
2  58114990   LIVE       2023-09-01 2040-07-30  NONCORE8          
3  58113435   LIVE       2023-09-01 2028-08-01  NONCORE3          
4  58109518   LIVE       2023-09-01 2033-05-25  ALGO2             

            INSTRUMENT             ISIN    ISSUER  \
0  SPGB521031 1.9 REGS  ES0000012K46      ESPAGNE   
1      FRTR321025 5.75  FR0000187635       TREPUB   
2       SPGB400730 4.9  ES00000120N0      ESPAGNE   
3  BTPS280801 3.8 REGS  IT0005548315     TRESORIT   
4   FRTR330525 3. REGS  FR001400H7V7       TREPUB   

                ISSUER_FULLNAME   NOMINAL CURRENCY BUY_SELL COUNTERPART  \
0             ROYAUME D ESPAGNE  100000.0      EUR        S    DWSXTDT9   
1  DIRECTION GENERALE DU TRESOR   12747.0      EUR        S    PICX1817   
2             ROYAUME D ESPAGNE    7000.0      EUR   

AttributeError: 'NoneType' object has no attribute 'head'

In [31]:
# Check for NaN values in critical columns
print(processed_trade_source['ISSUER'].isna().sum())
print(processed_trade_source_scope['ISSUER'].isna().sum())

# Verify that 'Period' columns are correctly assigned
print(processed_trade_source['Period'].unique())
print(esma_si_df['Period'].unique())


0
0


KeyError: 'Period'

Generate the Report:



In [32]:
# Specify the output directory (can be a temporary directory)
output_dir = 'path_to_output_directory'  # Replace with your desired path

# Instantiate ReportGenerator
report_generator = ReportGenerator(output_dir)

# Generate the report
report = report_generator.generate_report(
    esma_si_df=esma_si_df,
    trade_source=processed_trade_source,
    trade_source_scope=processed_trade_source_scope,
    result_df=result_df,
    issuer_review=issuer_review,
    all_periods=data_processor.all_periods
)

# Display the report
print(report)


TypeError: can only join an iterable

Example: Total Trades per Period



In [33]:
import matplotlib.pyplot as plt

# Total trades per period in Trade_Source
trade_counts = processed_trade_source['Period'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
trade_counts.plot(kind='bar')
plt.title('Total Trades per Period')
plt.xlabel('Period')
plt.ylabel('Number of Trades')
plt.show()


KeyError: 'Period'

Example: SI Scores per Issuer



In [34]:
# Ensure 'Total SI Score' is calculated
issuer_review['Total SI Score'] = issuer_review.filter(regex='SI Score').sum(axis=1)

# Top 10 issuers by SI Score
top_issuers = issuer_review.nlargest(10, 'Total SI Score')

plt.figure(figsize=(10, 6))
plt.barh(top_issuers['ISSUER'], top_issuers['Total SI Score'])
plt.title('Top 10 Issuers by Total SI Score')
plt.xlabel('Total SI Score')
plt.ylabel('Issuer')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest score on top
plt.show()


AttributeError: 'NoneType' object has no attribute 'filter'